In [0]:
from google.colab import drive
import pandas as pd
drive.mount('/topics_data/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /topics_data/


In [0]:
train = pd.read_csv("/topics_data/My Drive/Colab Notebooks/topics_data/train.tsv", sep='\t', lineterminator = '\n', header = None).fillna(0)

In [0]:
train.head()

,0,1,2,3
0,0,Краткая история российско-американских отношен...,Когда «Лишь бы не было войны» сменилось мемом ...,"2,7,10,29,50"
1,1,Видео: Квартира среднестатистического китайца,"Практически все уже знают, что Китай - произв...","79,95"
2,2,Дни Украины сочтены!,"Не смотря на столь громкий заголовок я, до ...","2,10,71"
3,3,Плюсы и минусы жизни на Мальте,"Доброго времени суток дорогие читатели, сегод...",15
4,4,В СССР все были неравны,На фоне громких расследований Фонда по Борьбе ...,3


In [0]:
train.shape

(126025, 4)

In [0]:
targets = {i:[0]*100 for i in range(len(train))}

In [0]:
train[3] = train[3].fillna(0).astype(str)

In [0]:
for i in range(len(train)):
    for j in train[3][i].split(','):
        targets[i][int(j)] = 1

In [0]:
target_df = pd.DataFrame.from_dict(targets, columns = [i for i in range(100)], orient='index')

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from scipy.sparse import hstack
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

In [0]:
!pip install pymystem3

In [0]:
import nltk
nltk.download("stopwords")
#--------#

from nltk.stem.snowball import SnowballStemmer 

stemmer = SnowballStemmer("russian") 

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

#Create lemmatizer and stopwords list
mystem = SnowballStemmer("russian") 
russian_stopwords = stopwords.words("russian")

#Preprocess function
def preprocess_text(text):
    global j
    if j%10000 == 0:
        print(j)
    j+=1
    txt = text.lower().split(' ')
    tokens = [stemmer.stem(word) for word in txt]
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    text = " ".join(tokens)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
j = 0
title = train[1].apply(preprocess_text)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000


In [0]:
j = 0
content = train[2].astype(str).apply(preprocess_text)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000


In [0]:
vectorizer_topic = TfidfVectorizer(stop_words=russian_stopwords, ngram_range=(1,2))
vectorizer_content = TfidfVectorizer(stop_words=russian_stopwords, ngram_range=(1,2))

In [0]:
X1 = vectorizer_topic.fit_transform(title)#exper_train[1])
X2 = vectorizer_content.fit_transform(content)#exper_train[2])

In [0]:
X1.shape, X2.shape

((126025, 478819), (126025, 17006781))

In [0]:
X = hstack([X1,X2])

In [0]:
X.shape

(126025, 17485600)

In [0]:
clf = []
f1 = []

In [0]:
for i in range(100):
    Y = target_df[i]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    clf_i = LogisticRegression(penalty='l1', random_state=241, C=2).fit(X_train, y_train)
    y_pred = clf_i.predict(X_test)
    rec_cur = metrics.recall_score(y_test, y_pred)
    f1_i = metrics.f1_score(y_test, y_pred)
    clf.append(clf_i)
    f1.append(f1_i)
    print('\t log reg l1', i ,'recall = ', rec_cur, 'f1 = ', f1_i)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [0]:
np.mean(f1)

In [0]:
test = pd.read_csv("/topics_data/My Drive/Colab Notebooks/topics_data/test.tsv", sep='\t', lineterminator = '\n', header = None).fillna('')

In [0]:
test.head()

In [0]:
test.shape

In [0]:
preprocess_text_title = test[1].apply(preprocess_text)

In [27]:
preprocess_text_content = test[2].apply(preprocess_text)

160000
170000
180000


In [0]:
X1 = vectorizer_topic.transform(preprocess_text_title)
X2 = vectorizer_content.transform(preprocess_text_content)
X_test = hstack([X1,X2])

In [38]:
result = []
for i in range(100):
    if i %10 == 0:
        print(i)
    y_pred = []
    y_proba = clf[i].predict_proba(X_test)
    y_pred = np.zeros_like(y_proba.T[0])
    for j, el in enumerate(y_proba.T[1]):
        if el > 0.25:
            y_pred[j] = 1
    result.append(np.array(y_pred))

0
10
20
30
40
50
60
70
80
90


In [0]:
result = np.array(result).T

In [0]:
topics_pred = []
for i in range(len(result)):
    topics_pred.append([])
    for j in range(len(result[i])):
        if result[i][j] == 1:
            topics_pred[i].append(j)
    if len(topics_pred[i]) == 0:
      topics_pred[i].append(0)
    topics_pred[i] = np.unique(topics_pred[i])

In [0]:
f = open('/topics_data/My Drive/Colab Notebooks/topics_data/submission.tsv', 'w')

In [0]:
ids = test[0]
for i, el in enumerate(topics_pred):
    f.write(str(ids[i]))
    f.write('\t')
    for j in range(len(el)-1):
        f.write(str(el[j]) + ',')
    f.write(str(el[len(el)-1]) + '\n')
f.close()